In [1]:
from gensim import corpora,models,similarities
from sklearn.datasets import fetch_20newsgroups as getData
from sklearn.model_selection import train_test_split
from collections import defaultdict
import re
import numpy as np

In [2]:
corpus = getData(subset='train', remove=('headers', 'footers', 'quotes'))
X = corpus.data
y = corpus.target
y_names = corpus.target_names

In [6]:
print(X[0],len(X),y_names[y[0]])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail. 11314 rec.autos


In [7]:
type(X[0])

str

In [8]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,test_size=0.1,shuffle=True)

In [9]:
stoplist = set('for a of the and to in'.split(' '))
texts = [[word for word in re.split('\W+',doc.lower()) if word not in stoplist]
         for doc in Xtrain]

In [11]:
print(len(texts))

10182


In [12]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

cutOffValue = 10
processed_corpus = [[token for token in text if frequency[token] > cutOffValue] 
                    for text in texts]

In [13]:
dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary(12401 unique tokens: ['', '38', 'an', 'animal', 'are']...)


In [14]:
print(dictionary.token2id)

{'': 0, '38': 1, 'an': 2, 'animal': 3, 'are': 4, 'as': 5, 'ask': 6, 'at': 7, 'ball': 8, 'be': 9, 'been': 10, 'bullets': 11, 'but': 12, 'by': 13, 'can': 14, 'capable': 15, 'certain': 16, 'circumstances': 17, 'controlled': 18, 'dangerous': 19, 'do': 20, 'enough': 21, 'eye': 22, 'finally': 23, 'fired': 24, 'from': 25, 'gas': 26, 'general': 27, 'have': 28, 'i': 29, 'ie': 30, 'intended': 31, 'is': 32, 'killed': 33, 'launcher': 34, 'like': 35, 'load': 36, 'looks': 37, 'low': 38, 'mother': 39, 'not': 40, 'now': 41, 'only': 42, 'or': 43, 'out': 44, 'past': 45, 'people': 46, 'put': 47, 'really': 48, 'round': 49, 'rubber': 50, 'seem': 51, 'shotguns': 52, 'smaller': 53, 'someone': 54, 'something': 55, 'stopping': 56, 'tear': 57, 'that': 58, 'them': 59, 'there': 60, 'they': 61, 'thing': 62, 'truly': 63, 'under': 64, 'understand': 65, 'use': 66, 'velocity': 67, 'very': 68, 'wants': 69, 'warned': 70, 'who': 71, 'with': 72, 'you': 73, 'your': 74, 'about': 75, 'amongst': 76, 'background': 77, 'basic':

In [15]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

In [16]:
print(bow_corpus[100])

[(0, 2), (4, 3), (5, 1), (29, 1), (35, 1), (40, 1), (58, 2), (62, 1), (75, 1), (185, 1), (206, 2), (336, 1), (442, 1), (445, 1), (781, 1), (950, 1), (951, 1), (1162, 1), (1224, 1), (1777, 1), (2147, 1), (2229, 1), (2378, 1), (2525, 1), (2527, 1), (3111, 1), (3318, 1), (3319, 1), (3320, 1), (3321, 1)]


In [17]:
model = models.TfidfModel(bow_corpus)
#model = models.LsiModel(bow_corpus)
#model = models.LdaModel(bow_corpus)

In [18]:
index = similarities.SparseMatrixSimilarity(model[bow_corpus],num_features=len(dictionary))

In [22]:
t = np.random.randint(len(Xtest))
query_document = re.split('\W+',Xtest[t].lower())
query_bow = dictionary.doc2bow(query_document)
sims = index[model[query_bow]]
docNumber = sorted(enumerate(sims), key=lambda x: x[1], reverse=True)[0][0]
print('Predicted:',y_names[ytrain[docNumber]])
print('Ground Truth:',y_names[ytest[t]])

Predicted: rec.autos
Ground Truth: sci.electronics
